# Geocoding and Web Mapping

Getting Started with Python

## Import statements

We are going to use [geopy](https://geopy.readthedocs.io/en/stable/) to interface with the mapbox geocoder and the [folium](http://python-visualization.github.io/folium/) to create some simple webmaps.

In [ ]:
from geopy.geocoders import MapBox
import folium
print('Import Successful')

## Using the geocoder
First, you must find your [access token](https://account.mapbox.com/access-tokens/).  Copy and paste it into the code below.

In [ ]:
access_token=""

if access_token == "":
    print('Enter your access token to continue')
else:
    geolocator = MapBox(api_key=access_token)
    print('Mapbox Goelocator Loaded')

## A quick test

Lets try a simple example first!  Type any address here and see what comes up!

In [ ]:
Location = "1984 West Mall Vancouver BC"
UBC_Geography = geolocator.geocode(Location)

print(UBC_Geography)
print(UBC_Geography.latitude,UBC_Geography.longitude)

## Displaying our result on a web map

In [ ]:
Map1 = folium.Map(
    location=[UBC_Geography.latitude,UBC_Geography.longitude],
    zoom_start=11,
)

point=folium.CircleMarker(
        location=[UBC_Geography.latitude,UBC_Geography.longitude],
        radius=10,
        popup='UBC Geography',
        fill_opacity = 1,
        fill=True,
        color='black',
        line_weight=.35,
        fill_color='blue'
)

point.add_to(Map1)

Map1

## Chaning our basemap and zoom level

In [ ]:
Map2 = folium.Map(
    location=[UBC_Geography.latitude,UBC_Geography.longitude],
    zoom_start=3,
    tiles='Stamen Terrain'
)

point.add_to(Map2)

Map2

## Using Mapbox satelite tiles

You can use your Mapbox account to access their satellite basemap layer.

In [ ]:
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(access_token)
Map3 = folium.Map(location=[UBC_Geography.latitude,UBC_Geography.longitude],
                 zoom_start=12,
                 tiles=tileurl, 
                 attr='Mapbox')

point.add_to(Map3)

Map3

## Using functions to avoid repetition

If we have multiple points to map, it would be redundant to type out the same command repeatedly.  We can create a [function](https://www.w3schools.com/python/python_functions.asp) to take some inputs and repeat the task for us.

In [ ]:
# "def" defines our function "plot_point()", which takes five "arguments":
    # Map: The map you're working with
    # X & Y: lattitude & longitude
    # Popup_Text: What do we want the popup to say?
    # Color: We'll set a defualt, but we can override with what colour do we want
        # Everything else will remain the same for every point, so we can set them as default values
    # Defaults can be over written by assigning them anoter value
def plot_point(Map,X,Y,Popup_Text,Color='red',Radius=5,Opacity=.75,LineColor='black',LineWidth=.15):
    folium.CircleMarker(
        # The coordiatnates
        location=[X,Y],
        # Text description
        popup=Popup_Text,
        # sets the fill color for the point
        fill_color=Color,
        # Size of the marker
        radius=Radius,
        # Opacity of the circle
        fill_opacity = Opacity,
        # Sets the line color for the edge
        color=LineColor,
        # Width of the border line
        line_weight=LineWidth,
    ).add_to(Map)
    
Map4 = folium.Map(
location=[0,0],
zoom_start=2,
)

for city in ['Cairo EG','London UK','Toronto CA','Sao Palo BR']:
    Result = geolocator.geocode(city)
    point = plot_point(Map4,Result.latitude,Result.longitude,city)


Map4

## Reverse geocoding

You can also go "backwards", starting with coordinates and getting an address *provided its somewhere with an address*

The second query returns "None"

* Any ideas why?

In [ ]:
Location = ['49.265646','-123.256218']
Reverse_1 = geolocator.reverse(Location)
print(Reverse_1)

# Note the different latitude
Location = ['-49.265646','-123.256218']
Reverse_2 = geolocator.reverse(Location)
print(Reverse_2)

## Watch out for typos

The functionality is fairly robust, but typos can cause errors in your results

In [ ]:
Map5 = folium.Map(location=[UBC_Geography.latitude,UBC_Geography.longitude],
                 zoom_start=14,
                 tiles=tileurl, 
                 attr='Mapbox')

for typo in ["2710 Fraser St. Vancouver BC V5P 3V4",
             "2711 Fraser St. Vancouver BC V5T 3V7",
             "2710 Faster Vancouver DC"]:
    UBC_Geography = geolocator.geocode(typo)

    point = plot_point(Map5,UBC_Geography.latitude,UBC_Geography.longitude,UBC_Geography)

    print('Search Term: ', typo)
    print('Result: ', UBC_Geography)
    print()

Map5

## A Note on Ambiguity

**Ambiguity**: the quality of being open to more than one interpretation; inexactness.

It is important to be specific when submitting queries.  The more information the geocoder has to work with, the more accurate your result.

In [ ]:
ExampleMap = folium.Map(
    location=[0,0],
    zoom_start=2,
    tiles='Stamen Toner'
)

for search in ['Surrey','Surrey BC','Victoria','Victoria BC']:
    UBC_Geography = geolocator.geocode(search)
    plot_point(ExampleMap,UBC_Geography.latitude,UBC_Geography.longitude,UBC_Geography,Radius=15)
    print('Search Term: ', UBC_Geography)
    print('Result: ', UBC_Geography)
    print()
    
ExampleMap

# Plotting Polygons

## Importing and Reformatting a Shapefile

- Below is a .zip file of census sub-divisions from Simply Analytics.
- We need to unzip it and inspect the metadata

In [ ]:
import zipfile
import pandas as pd
Shape_file='SimplyAnalytics_Shapefiles_2021-11-07_04_12_18_e28bde0952164f8aa648e288cab05f1b'
with zipfile.ZipFile('data/'+Shape_file+'.zip', 'r') as zip_ref:
    zip_ref.extractall('data/Census/')
# print()
meta_data = pd.read_csv('data/Census/variable_names.txt',header=None,sep='#')
print(meta_data.values)

## Reformat the shapefile

Folium needs the data in a different format, known a as a geojson file.  We can convert the file with [geopandas](https://geopandas.org/en/stable/) after doing some calculations.  Geopandas is a spatial extension for pandas.

In [ ]:
import geopandas as gpd

# the .read_file() function reads shapefiles
BC_Sub_Div = gpd.read_file('data/Census/'+Shape_file+'.shp')
print(BC_Sub_Div.crs)

## Note - This is the terminology used by the census
## It can be found in the variable_name file that comes with the download
BC_Sub_Div = BC_Sub_Div.rename(columns={
'VALUE0': 'Indigenous Identity',
'VALUE1': 'Population',
'VALUE2': 'Visible Minority'
})

BC_Sub_Div['Indigenous_Pct'] = (BC_Sub_Div['Indigenous Identity']/BC_Sub_Div['Population']*100).round(0)


BC_Sub_Div=BC_Sub_Div.fillna(0)
# .to_file() saves our data to the specified format
print('data Converted')
BC_Sub_Div.to_file("data/BC_Sub_Div.json", driver = "GeoJSON")
BC_Sub_Div.head()
# BC_Sub_Div['Indigenous Identity'].describe()

## Making a Choropleth Map

Foium's [Choropleth](https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps) allows us to quickly display color-coded polygons.


In [ ]:
BC_coords = geolocator.geocode('BC, Canada')
BC_Map = folium.Map(
    location=[BC_coords.latitude,BC_coords.longitude],
    zoom_start=5,
    tiles='Stamen Toner'
)


folium.features.Choropleth('data/BC_Sub_Div.json',
                           # It will match the geometry data up with a pandas or geopandas dataframe
                            data=BC_Sub_Div,
                            columns=['spatial_id','Indigenous_Pct'],
                           # They key in the GeoJSON file to match by
                            key_on='feature.properties.spatial_id',
                           # If we define bins, it will split where we tell it to
#                             bins = [0,10,20,55],
                            fill_color='YlOrRd',
                            fill_opacity = 1,
                            smooth=2,
                           # The layer name
                            name='Indigenous Population %',
                           # The legend label
                            legend_name='Indigenous Population %',
                           # Whether it shows up by default or must be turned on with the legend
                            show=True
                          ).add_to(BC_Map)


folium.LayerControl().add_to(BC_Map)
BC_Map

# Adding a tool tip

We can add a tooltip that allows us to display information when we hover over a polygon.

In [ ]:
import branca.colormap as cm


BC_Map2 = folium.Map(
    location=[BC_coords.latitude,BC_coords.longitude],
    zoom_start=5,
    tiles='Stamen Toner'
)

colormap = cm.LinearColormap(['#f5f8fa','#0f91f5'],
                             vmin=BC_Sub_Div['Indigenous_Pct'].min(),
                             vmax=BC_Sub_Div['Indigenous_Pct'].max())
colormap = colormap.to_step(n=4)#
colormap.caption = 'Indigenous Population  %'
colormap.add_to(BC_Map2)

folium.GeoJson(
    'data/BC_Sub_Div.json',
    name='Indigenous Population %',
    smooth_factor=1.75,
    style_function = lambda x:{'color':'black',
                               "weight": 1,
                               "fillOpacity": 1,
                              'fillColor':colormap(x['properties']['Indigenous_Pct'])
                              },
    tooltip=folium.features.GeoJsonTooltip(fields=['Population',
                                                   'Visible Minority',
                                                   'Indigenous Identity',
                                                   'Indigenous_Pct'],
                                           aliases=['Total Population, 2021',
                                                    'Visible Minority Population, 2021',
                                                    'Indigineous Population, 2021',
                                                    'Percent Indigineous, 2021',]
                                          ),
    show = True
).add_to(BC_Map2)



folium.LayerControl().add_to(BC_Map2)
BC_Map2